In [ ]:
lt = []

def shift_dict(mydt,newKey ,lt, keep_len = 10):
    lt.append(newKey)
    if len(lt) >keep_len:
        headKey = lt.pop(0)
        del mydt[headKey]
    return mydt, lt
from collections import defaultdict
mydt = defaultdict(str)
lt = []
for i in range(50):
    mydt[f'{i}-key'] = f'value {i}'
    shift_dict(mydt,f'{i}-key' ,lt, keep_len = 10)
    
print(mydt)

In [1]:
seq = "MPOQGLAYPVPTLIYHITHLNNLQGILQRGGLLPYSQRPPTQONVAYGHIQAHRAQVVVPVGPRGKLHDYVPFYFCPRSPMLYAIHTQQTDYQGDQRPILHLVSSAQKVAEARIPFVETDRHAVQYVCFFHKLEHLKALDWQAIQASYWANVREKKOAEFLVKDFFPWELVEEIGVIDKTIQAQVESILAQFPDLHHPPVRVRRSWYYK"
restypes = [ 
     'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 
     'S', 'T', 'W', 'Y', 'V' 
 ] 

In [3]:
for idex, value in enumerate(seq):
    if value not in  restypes:
        print(idex, value)

2 O
42 O
156 O


In [13]:
import json
Fi = "/dat1/nbt2/proj/22-cas5/work/blast/Cas5_Csy2.json"
f = open(Fi, 'r')
blast_json = json.load(f)
f.close()

In [16]:
dt = blast_json["BlastOutput2"][0]
reprot = dt['report']
params = reprot['params']
results = reprot['results']
iterations = results['iterations']

In [ ]:
data = []
E_VALUE_THRESH = 0.001
for iter in iterations:
    search = iter['search']
    hits = search['hits']
    num_matches = len(search['hits'])
    print("num_matches:",num_matches)
    query_length = search["query_len"]
    query = search["query_id"]
    for hit_ in hits:
        
        title = hit_['description'][0]["title"]
        title_split = title.split()
        title_id = title_split[0]
        title_seq_id = title_split[1]
        title_desc = title.split(']')[0] + ']'
        length = hit_["len"]
        # print(title_seq_id)
        align = ''
        for hsp in hit_["hsps"]:
            desc = ''
            if hsp['evalue'] < E_VALUE_THRESH:
                score = hsp["score"]
                bits = hsp["bit_score"]
                identities = hsp[ "identity"]
                query_start = hsp["query_from"]
                sbjct_start = hsp["hit_from"]
                sbjct_to = hsp["hit_to"]
                e_value = hsp["evalue"]
                # query = hsp.query
                # match = hsp.match
                # sbjct = hsp.sbjct
                align_length = hsp["align_len"]
                gaps = hsp["gaps"]
                # print(hsp)
                # break
                ident = round(identities / align_length * 100, 2)
                gap_per = round(gaps / align_length * 100, 2)
                query_cover = round(align_length / query_length * 100, 2)
                
                align += 'hsp: ' + str(sbjct_start) + ' to ' + str(
                    sbjct_start + align_length - 1) + ','
                    
                align += f" Score = {str(score)}  bits ({str(hsp.bits)}), Expect = {str(e_value)} " 
                align +=f"Identities = {identities}/{align_length} ({ident}%), Gaps = {hsp.gaps}/{align_length} ({gap_per}%)\n\n"
                #Positives = {hsp.gaps}/{align_length} (75%)
                loops = round(align_length // 60 + 1)
        #        
                for i in range(0, loops):
                    align += "Query %8s  %s %s\n" % (str(query_start), query[(i * 60):((i + 1) * 60)],
                                                        str(query_start + len(query[(i * 60):((i + 1) * 60)]) - 1))
                    align += "      %8s  %s\n" % ('', match[(i * 60):((i + 1) * 60)])
                    align += "Sbjct %8s  %s %s\n\n" % (str(sbjct_start), sbjct[(i * 60):((i + 1) * 60)], str(
                        sbjct_start + len(sbjct[(i * 60):((i + 1) * 60)]) - 1))
                    query_start += 60
                    sbjct_start += 60
                
                align += "\n\n"

                dt = {
                "target":title_seq_id,
                "query": query,
                "query_covery": query_cover,
                "target_len": align_length,
                "E_value": e_value,
                "score": float(score),
                "domain": 1,
                "domain_count": 1,
                "acc": float(ident),
                'desc': title_desc ,
                "pairwise":[align]
                }
                data.append(dt)

In [4]:
f2 = "/dat1/nbt2/proj/21-prot/web/data/res/blast/07fe80bf-fcc4-4138-b449-5ce678f19036/BLASTP.json"
f = open(f2, 'r')
dt2 = json.load(f)
f.close()